In [2]:
# (The NHL and the NHL Shield are registered trademarks of the 
# National Hockey League. NHL and NHL team marks are the property of 
# the NHL and its teams. © NHL 2023. All Rights Reserved.)

import pandas as pd
import datetime
import os
import requests
import json
import matplotlib.pyplot as plt
import numpy as np
import hvplot.pandas
import geopandas as gpd
import seaborn as sns
import scipy.stats as st
from sqlalchemy import create_engine
from random import randint
from time import sleep
from scipy.stats import pearsonr, mannwhitneyu



In [3]:
# Get Stanely Cup Winning and Losing teams since 1927
# Code by Ron Brennan

url = "https://en.wikipedia.org/wiki/List_of_Stanley_Cup_champions"

stanleycup_champs = pd.read_html(url)

stanleycup_byyear_df = stanleycup_champs[2]
stanleycup_byyear_df.head()

,Year,Winning team,Coach,Games,Losing team,Coach.1,Winning goal
0,1927,"Ottawa Senators (C) (5, 4–1)",Dave Gill,2–0,"Boston Bruins (A) (1, 0–1)",Art Ross,"Cy Denneny (7:30, second)"
1,1928,"New York Rangers (A) (1, 1–0)",Lester Patrick-playing,3–2,"Montreal Maroons (C) (2, 1–1)",Eddie Gerard,"Frank Boucher (3:35, third)"
2,1929,"Boston Bruins (A) (2, 1–1)",Art Ross,2–0,"New York Rangers (A) (2, 1–1)",Lester Patrick,"Bill Carson (18:02, third)"
3,1930,"Montreal Canadiens (C) (6, 3–2)",Cecil Hart,2–0,"Boston Bruins (A) (3, 1–2)",Art Ross,"Howie Morenz (1:00, second)"
4,1931,"Montreal Canadiens (C) (7, 4–2)",Cecil Hart,3–2,"Chicago Black Hawks (A) (1, 0–1)",Dick Irvin,"Johnny Gagnon (9:59, second)"


In [4]:
# Find Winning Teams from 1991-2023
# Code by Ron Brennan

stanleycup_winner_cleaned_df = stanleycup_byyear_df.loc[stanleycup_byyear_df['Year'].between(1992,2023)]
stanleycup_winner_cleaned_df = stanleycup_winner_cleaned_df[["Year", "Winning team"]]

stanleycup_winner_cleaned_df.reset_index(drop=True, inplace=True)

stanleycup_winner_cleaned_df["Year"] = (stanleycup_winner_cleaned_df["Year"] -1).map(str) + (stanleycup_winner_cleaned_df["Year"]).map(str)
stanleycup_winner_cleaned_df["Winning team"] = stanleycup_winner_cleaned_df["Winning team"].str.split('(').str[0]

stanleycup_winner_cleaned_df.drop(stanleycup_winner_cleaned_df[(stanleycup_winner_cleaned_df['Year'] == "20042005")].index, inplace=True)
stanleycup_winner_cleaned_df.rename(columns={"Winning team":"Team Name"}, inplace=True)
stanleycup_winner_cleaned_df["Team Name"] = stanleycup_winner_cleaned_df["Team Name"].str.strip()

# Change names 
stanleycup_winner_cleaned_df["Team Name"] = stanleycup_winner_cleaned_df["Team Name"].str.replace("Montreal Canadiens", "Montréal Canadiens")
stanleycup_winner_cleaned_df

,Year,Team Name
0,19911992,Pittsburgh Penguins
1,19921993,Montréal Canadiens
2,19931994,New York Rangers
3,19941995,New Jersey Devils
4,19951996,Colorado Avalanche
5,19961997,Detroit Red Wings
6,19971998,Detroit Red Wings
7,19981999,Dallas Stars
8,19992000,New Jersey Devils
9,20002001,Colorado Avalanche


In [5]:
# Get Team Name and associated with its ID in the NHL API JSON request

teamID_url = "https://statsapi.web.nhl.com/api/v1/teams"
   
response = requests.get(teamID_url)

response_json = response.json()
team_info = {}

for team in response_json["teams"]:
    team_id = team["id"]
    team_name = team["name"]
    team_info[team_id] = team_name
   
team_info_df = pd.DataFrame.from_dict(team_info, orient="index", columns=["Team Name"])
team_info_df.reset_index(inplace=True)
team_info_df.columns = ["Team ID", "Team Name"]

team_info_df

,Team ID,Team Name
0,1,New Jersey Devils
1,2,New York Islanders
2,3,New York Rangers
3,4,Philadelphia Flyers
4,5,Pittsburgh Penguins
5,6,Boston Bruins
6,7,Buffalo Sabres
7,8,Montréal Canadiens
8,9,Ottawa Senators
9,10,Toronto Maple Leafs


In [6]:
# Merge dataframes to associate the Winning Team and Season won with the Team ID

stanleycup_winning_merge_byname_ID = pd.DataFrame.merge(stanleycup_winner_cleaned_df,team_info_df, on='Team Name', how='left')
stanleycup_winning_merge_byname_ID

,Year,Team Name,Team ID
0,19911992,Pittsburgh Penguins,5
1,19921993,Montréal Canadiens,8
2,19931994,New York Rangers,3
3,19941995,New Jersey Devils,1
4,19951996,Colorado Avalanche,21
5,19961997,Detroit Red Wings,17
6,19971998,Detroit Red Wings,17
7,19981999,Dallas Stars,25
8,19992000,New Jersey Devils,1
9,20002001,Colorado Avalanche,21


In [7]:
# Get Roster for each Winning team per Season.  
# Will use this code

player_list = []
base_url = "https://statsapi.web.nhl.com/api/v1/teams/{}?expand=team.roster&season={}"
for index, row in stanleycup_winning_merge_byname_ID.iterrows():
    team_ids = row ["Team ID"]
    # print(team_ids)
    year_cl= row ["Year"]
    # print(year_cl)
    url = base_url.format(team_ids, year_cl)
    # print(url)
    response = requests.get(url)
    data = response.json()
    
    for player in data["teams"][0]["roster"]["roster"]:
        
        person = player["person"]["fullName"]
        # Create dictionary
        player_id = player["person"]["id"]
        # Printing for testing purposes
        #print("Name:", person)
        #print("ID:", player_id)
        player_list.append({
            "Name":person,
            "Player ID":player_id,
            "Team ID": team_ids,
            "Season":year_cl
        })
winning_df=pd.DataFrame(player_list)
winning_df.tail(30)
     

,Name,Player ID,Team ID,Season
1011,Byron Froese,8475278,54,20222023
1012,Mark Stone,8475913,54,20222023
1013,William Karlsson,8476448,54,20222023
1014,Jonathan Marchessault,8476539,54,20222023
1015,Chandler Stephenson,8476905,54,20222023
1016,Teddy Blueger,8476927,54,20222023
1017,Ben Hutton,8477018,54,20222023
1018,Shea Theodore,8477447,54,20222023
1019,William Carrier,8477478,54,20222023
1020,Ivan Barbashev,8477964,54,20222023


In [8]:
   
# Player Profile - Winning Team per Season
# Code by Ron Brennan

player_info_url = "https://statsapi.web.nhl.com/api/v1/people/{}"
api_base_url = 'https://api.api-ninjas.com/v1/city?'
api_key = 'nVSl3APKDcl75e5gd1nmBg==GekBJP6bGG8nSITj'
player_info = []
player_stats = []

def get_lat_long(city):
            
            latlong = []

            api_url = api_base_url + 'name={}'.format(city)
            response_latlong = requests.get(api_url, headers={'X-Api-Key': api_key})
            
            if response_latlong.status_code == requests.codes.ok:
                data_list = response_latlong.json()
                if data_list:
                    data = data_list[0]  # Assuming the API returns a list of results and we take the first one
                    latitude = data.get('latitude')
                    longitude = data.get('longitude')
                    latlong.append(latitude)
                    latlong.append(longitude)
                else:
                    latitude = None
                    longitude = None
                    latlong.append(latitude)
                    latlong.append(longitude)

                    
            else:
                latitude = None
                longitude = None
                latlong.append(latitude)
                latlong.append(longitude)                   
            
            return latlong

for index, row in winning_df.iterrows():
    player_nhl_id = row["Player ID"]
    winning_season = row["Season"]
    url = player_info_url.format(player_nhl_id)
    response = requests.get(url)
    data = response.json()

    for players in data["people"]:
            player_id = players["id"]
            person = players["fullName"]
            player_firstName = players["firstName"]
            player_lastName = players["lastName"]
            player_birthDate = players["birthDate"]
            player_birthCity = players["birthCity"]
            try:
                player_lat_long = get_lat_long(player_birthCity)
            except:
                pass  
               
            player_birthCountry = players["birthCountry"]
            try:
                player_birthStateProvince = players["birthStateProvince"]
            except Exception as e:
                player_birthStateProvince = 'N/A'     
            player_nationality = players["nationality"]
            player_height = players["height"]
            player_weight = players["weight"]
            try:
                player_shootsCatches = players["shootsCatches"]
            except Exception as e:
                player_shootsCatches = 'L'     
            player_primaryPositionCode = players["primaryPosition"]["code"]

            player_info.append({
                "player_id":player_id,
                "player_name":person,
                "player_firstName":player_firstName,
                "player_lastName":player_lastName,
                "player_birthDate":player_birthDate,
                "player_birthCity":player_birthCity,
                "player_birthCity_Lat":player_lat_long[0],
                "player_birthCity_Long":player_lat_long[1],
                "player_birthCountry":player_birthCountry,
                "player_birthStateProvince": player_birthStateProvince,
                "player_nationality":player_nationality,
                "player_height":player_height,
                "player_weight":player_weight,
                "player_shootsCatches":player_shootsCatches,
                "player_primaryPositionCode":player_primaryPositionCode,
                "season": int(winning_season)
            })
                  
            
player_information_df = pd.DataFrame(player_info)
player_information_df = player_information_df.drop_duplicates(subset='player_id', keep="first")
player_information_sorted_df = player_information_df.sort_values(by=['player_id'])

goalie_df = player_information_sorted_df.loc[player_information_sorted_df['player_primaryPositionCode'] == 'G']

player_information_sorted_df.drop(['season'], axis=1, inplace = True)
player_information_sorted_df.to_csv("./schema/player_information_utf-8.csv", index=False, encoding = 'utf-8-sig') 
player_information_sorted_df.to_csv("./schema/player_information.csv", index=False) 
player_information_sorted_df.head()


,player_id,player_name,player_firstName,player_lastName,player_birthDate,player_birthCity,player_birthCity_Lat,player_birthCity_Long,player_birthCountry,player_birthStateProvince,player_nationality,player_height,player_weight,player_shootsCatches,player_primaryPositionCode
107,8444919,Tommy Albelin,Tommy,Albelin,1964-05-21,Stockholm,59.3294,18.0686,SWE,N/A,SWE,"6' 2""",195,L,D
71,8444945,Glenn Anderson,Glenn,Anderson,1960-10-02,Vancouver,49.2500,-123.1000,CAN,BC,CAN,"6' 1""",190,L,R
403,8445000,Dave Andreychuk,Dave,Andreychuk,1963-09-29,Hamilton,43.2567,-79.8692,CAN,ON,CAN,"6' 4""",225,R,L
33,8445275,Tom Barrasso,Tom,Barrasso,1965-03-31,Boston,42.3188,-71.0846,USA,MA,USA,"6' 3""",210,R,G
269,8445386,Ed Belfour,Ed,Belfour,1965-04-21,Carman,NaN,NaN,CAN,MB,CAN,"6' 0""",214,L,G


In [ ]:
player_information_df.head(10)
goalie_df.head(10)

,player_id,player_name,player_firstName,player_lastName,player_birthDate,player_birthCity,player_birthCountry,player_birthStateProvince,player_nationality,player_height,player_weight,player_shootsCatches,player_primaryPositionCode,season
0,8445629,Phil Bourque,Phil,Bourque,1962-06-08,Chelmsford,USA,MA,USA,"6' 1""",196,L,L,19911992
1,8446013,Jay Caufield,Jay,Caufield,1960-07-17,Philadelphia,USA,PA,USA,"6' 4""",237,R,R,19911992
2,8446050,Jeff Chychrun,Jeff,Chychrun,1966-05-03,LaSalle,CAN,QC,CAN,"6' 4""",215,R,D,19911992
3,8446117,Paul Coffey,Paul,Coffey,1961-06-01,Weston,CAN,ON,CAN,"6' 0""",205,L,D,19911992
4,8446305,Jeff Daniels,Jeff,Daniels,1968-06-24,Oshawa,CAN,ON,CAN,"6' 1""",200,L,L,19911992
5,8446424,Gord Dineen,Gord,Dineen,1962-09-21,Quebec City,CAN,QC,CAN,"6' 0""",195,R,D,19911992
6,8446715,Bob Errey,Bob,Errey,1964-09-21,Montréal,CAN,QC,CAN,"5' 11""",175,L,L,19911992
7,8446951,Ron Francis,Ron,Francis,1963-03-01,Sault Ste. Marie,CAN,ON,CAN,"6' 3""",200,L,C,19911992
8,8448029,Jiri Hrdina,Jiri,Hrdina,1958-01-05,Prague,CZE,N/A,CZE,"6' 0""",195,L,C,19911992
9,8448208,Jaromir Jagr,Jaromir,Jagr,1972-02-15,Kladno,CZE,N/A,CZE,"6' 3""",230,L,R,19911992


In [9]:
# Player Statistics - Winning Team (Skaters) per Season
# Code by Ron Brennan

player_stats_url = "https://statsapi.web.nhl.com/api/v1/people/{}/stats?stats=statsSingleSeason&season={}"
player_stats = []

winning_skaters_df = player_information_df.loc[player_information_df['player_primaryPositionCode'] != 'G']

for index, row in winning_skaters_df.iterrows():
     
        player_nhl_id = row["player_id"]
        season = row ["season"]
        player_birthyear_yyyy = row["player_birthDate"]
        win_season_yyyy = str(season)
        url = player_stats_url.format(player_nhl_id, season)
        response = requests.get(url)
        data_st = response.json()
        

        for player_st in data_st["stats"][0]["splits"]:
                player_season = player_st["season"]
                try:
                    player_timeOnIce = player_st["stat"]["timeOnIce"]
                except Exception as e:
                    player_timeOnIce = '-1'     
                player_assists = player_st["stat"]["assists"]
                player_goals = player_st["stat"]["goals"]
                player_pim = player_st["stat"]["pim"]
                player_shots= player_st["stat"]["shots"]
                player_games = player_st["stat"]["games"]
                try:
                    player_hits = player_st["stat"]["hits"]
                except Exception as e:
                    player_hits = '-1'
                player_powerPlayGoals = player_st["stat"]["powerPlayGoals"]
                player_powerPlayPoints = player_st["stat"]["powerPlayPoints"]
                try:
                    player_powerPlayTimeOnIce = player_st["stat"]["powerPlayTimeOnIce"]
                except Exception as e:
                    player_powerPlayTimeOnIce = '-1'
                try:
                    player_evenTimeOnIce = player_st["stat"]["evenTimeOnIce"]
                except Exception as e:
                    player_evenTimeOnIce = '-1'
                player_penaltyMinutes = player_st["stat"]["penaltyMinutes"]
                player_shotPct = player_st["stat"]["shotPct"]
                player_gameWinningGoals = player_st["stat"]["gameWinningGoals"]
                player_overTimeGoals = player_st["stat"]["overTimeGoals"]
                player_shortHandedGoals = player_st["stat"]["shortHandedGoals"]
                player_shortHandedPoints = player_st["stat"]["shortHandedPoints"]
                try:
                    player_blocked = player_st["stat"]["blocked"]
                except Exception as e:
                    player_blocked = '-1'
                player_plusMinus = player_st["stat"]["plusMinus"]
                player_points = player_st["stat"]["points"]
                try:
                    player_shifts = player_st["stat"]["shifts"]
                except Exception as e:
                    player_shifts = '-1'
                try:
                    player_timeOnIcePerGame = player_st["stat"]["timeOnIcePerGame"]
                except Exception as e:
                    player_timeOnIcePerGame = '-1'
                try:
                    player_evenTimeOnIcePerGame = player_st["stat"]["evenTimeOnIcePerGame"]
                except Exception as e:
                    player_evenTimeOnIcePerGame = '-1'
                try:
                    player_shortHandedTimeOnIcePerGame = player_st["stat"]["shortHandedTimeOnIcePerGame"]
                except Exception as e:
                    player_shortHandedTimeOnIcePerGame = '-1'
                try:
                    player_powerPlayTimeOnIcePerGame = player_st["stat"]["powerPlayTimeOnIcePerGame"]
                except Exception as e:
                    player_powerPlayTimeOnIcePerGame = '-1'

                player_season_year = win_season_yyyy[4:]
                player_birth_year = player_birthyear_yyyy[:4]
                winning_age = (int(player_season_year)) - (int(player_birth_year))
                
                player_stats.append({
                    "player_stats_id" : '', 
                    "player_id" : player_nhl_id, 
                    "season_id" : player_season,
                    "player_stats_age" : winning_age,
                    "player_stats_timeOnIce" : player_timeOnIce,
                    "player_stats_assists" : player_assists,
                    "player_stats_goals" : player_goals,
                    "player_stats_pim" : player_pim,
                    "player_stats_shots" : player_shots,
                    "player_stats_games" : player_games,
                    "player_stats_hits" : player_hits,
                    "player_stats_powerPlayGoals" : player_powerPlayGoals,
                    "player_stats_powerPlayPoints" : player_powerPlayPoints,
                    "player_stats_powerPlayTimeOnIce" : player_powerPlayTimeOnIce,
                    "player_stats_evenTimeOnIce" : player_evenTimeOnIce,
                    "player_stats_penaltyMinutes" : player_penaltyMinutes,
                    "player_stats_shotPct" : player_shotPct,
                    "player_stats_gameWinningGoals" : player_gameWinningGoals,
                    "player_stats_overTimeGoals" : player_overTimeGoals,
                    "player_stats_shortHandedGoals" : player_shortHandedGoals,
                    "player_stats_shortHandedPoints" : player_shortHandedPoints,
                    "player_stats_blocked" : player_blocked,
                    "player_stats_plusMinus" : player_plusMinus,
                    "player_stats_points" : player_points,
                    "player_stats_shifts" : player_shifts,
                    "player_stats_timeOnIcePerGame" : player_timeOnIcePerGame,
                    "player_stats_evenTimeOnIcePerGame" : player_evenTimeOnIcePerGame,
                    "player_stats_shortHandedTimeOnIcePerGame" : player_shortHandedTimeOnIcePerGame,
                    "player_stats_powerPlayTimeOnIcePerGame" : player_powerPlayTimeOnIcePerGame  
        })               
      
player_statistics = pd.DataFrame(player_stats)
player_statistics.to_csv("./schema/player_statistics_utf-8.csv", index=False, encoding = 'utf-8-sig')
player_statistics.to_csv("./schema/player_statistics.csv", index=False)
player_statistics.head()
      

,player_stats_id,player_id,season_id,player_stats_age,player_stats_timeOnIce,player_stats_assists,player_stats_goals,player_stats_pim,player_stats_shots,player_stats_games,...,player_stats_shortHandedGoals,player_stats_shortHandedPoints,player_stats_blocked,player_stats_plusMinus,player_stats_points,player_stats_shifts,player_stats_timeOnIcePerGame,player_stats_evenTimeOnIcePerGame,player_stats_shortHandedTimeOnIcePerGame,player_stats_powerPlayTimeOnIcePerGame
0,,8445629,19911992,30,-1,16,10,58,51,58,...,1,3,-1,-6,26,-1,-1,-1,-1,-1
1,,8446013,19911992,32,-1,0,0,175,16,50,...,0,0,-1,-6,0,-1,-1,-1,-1,-1
2,,8446050,19911992,26,-1,4,0,111,26,43,...,0,0,-1,-12,4,-1,-1,-1,-1,-1
3,,8446117,19911992,31,-1,58,11,87,232,64,...,0,2,-1,1,69,-1,-1,-1,-1,-1
4,,8446305,19911992,24,-1,0,0,0,0,2,...,0,0,-1,0,0,-1,-1,-1,-1,-1


In [9]:
# Player Statistics - Winning Team (Goalies) per Season
# Code by Ron Brennan

player_stats_url = "https://statsapi.web.nhl.com/api/v1/people/{}/stats?stats=statsSingleSeason&season={}"
goalie_stats = []

for index, row in goalie_df.iterrows():
    player_nhl_id = row["player_id"]
    season = row ["season"]
    player_birthyear_yyyy = row["player_birthDate"]
    win_season_yyyy = str(season)
    url = player_stats_url.format(player_nhl_id, season)
    response = requests.get(url)
    data_st = response.json()

    for goalie_st in data_st["stats"][0]["splits"]:
           
            player_season = goalie_st["season"]
            goalie_timeOnIce = goalie_st["stat"]["timeOnIce"]
            try:
                goalie_ot = goalie_st["stat"]["ot"]
            except Exception as e:
                 goalie_ot = '-1'
            goalie_shutouts = goalie_st["stat"]["shutouts"]
            try:
                goalie_ties = goalie_st["stat"]["ties"]
            except Exception as e:
                 goalie_ties = '-1'         
            goalie_wins = goalie_st["stat"]["wins"]
            goalie_losses = goalie_st["stat"]["losses"]
            goalie_saves = goalie_st["stat"]["saves"]
            try:
                goalie_powerPlaySaves = goalie_st["stat"]["powerPlaySaves"]
            except Exception as e:
                goalie_powerPlaySaves = '-1'
            try:
                goalie_shortHandedSaves = goalie_st["stat"]["shortHandedSaves"]
            except Exception as e:
                goalie_shortHandedSaves = '-1'
            try:
                goalie_evenSaves = goalie_st["stat"]["evenSaves"]
            except Exception as e:
                goalie_evenSaves = '-1'
            try:
                goalie_shortHandedShots = goalie_st["stat"]["shortHandedShots"]
            except Exception as e:
                goalie_shortHandedShots = '-1'
            try:
                goalie_evenShots = goalie_st["stat"]["evenShots"]
            except Exception as e:
                goalie_evenShots = '-1'
            try:
                goalie_powerPlayShots = goalie_st["stat"]["powerPlayShots"]
            except Exception as e:
                goalie_powerPlayShots = '-1'
            try:
                goalie_savePercentage = goalie_st["stat"]["savePercentage"]
            except Exception as e:
                goalie_savePercentage = '-1'
            try:
                goalie_goalAgainstAverage = goalie_st["stat"]["goalAgainstAverage"]
            except Exception as e:
                goalie_goalAgainstAverage = '-1'
            goalie_games = goalie_st["stat"]["games"]
            goalie_gamesStarted = goalie_st["stat"]["gamesStarted"]
            goalie_shotsAgainst = goalie_st["stat"]["shotsAgainst"]
            goalie_goalsAgainst = goalie_st["stat"]["goalsAgainst"]
            try:
                goalie_timeOnIcePerGame = goalie_st["stat"]["timeOnIcePerGame"]
            except Exception as e:
                goalie_timeOnIcePerGame = '-1'          
            try:
                goalie_powerPlaySavePercentage = goalie_st["stat"]["powerPlaySavePercentage"]
            except Exception as e:
                goalie_powerPlaySavePercentage = '-1'
            try:
                goalie_shortHandedSavePercentage = goalie_st["stat"]["shortHandedSavePercentage"]
            except Exception as e:
                goalie_shortHandedSavePercentage = '-1'
            try:
                goalie_evenStrengthSavePercentage = goalie_st["stat"]["evenStrengthSavePercentage"]
            except Exception as e:
                goalie_evenStrengthSavePercentage = '-1'


            player_season = win_season_yyyy[4:]
            player_birth_year = player_birthyear_yyyy[:4]
            winning_age = (int(player_season)) - (int(player_birth_year))
            goalie_stats.append({
                "goalie_stats_id" : '',
                "player_id" : player_nhl_id, 
                "season_id" : player_season,
                "goalie_stats_age" : winning_age,
                "goalie_stats_timeOnIce" : goalie_timeOnIce,
                "goalie_stats_goalie_ot" : goalie_ot,
                "goalie_stats_goalie_shutouts" : goalie_shutouts,
                "goalie_stats_goalie_ties" : goalie_ties,
                "goalie_stats_wins" : goalie_wins,
                "goalie_stats_goalie_losses" : goalie_losses,
                "goalie_stats_saves" : goalie_saves,               
                "goalie_stats_powerPlaySaves" : goalie_powerPlaySaves,
                "goalie_stats_powerPlayPoints" : goalie_shortHandedSaves,
                "goalie_stats_shortHandedSaves" : goalie_evenSaves,
                "goalie_stats_shortHandedShots" : goalie_shortHandedShots,
                "goalie_stats_evenShots" : goalie_evenShots,
                "goalie_stats_powerPlayShot" : goalie_powerPlayShots,
                "goalie_stats_savePercentage" : goalie_savePercentage,
                "goalie_stats_goalAgainstAverage" : goalie_goalAgainstAverage,
                "goalie_stats_games" : goalie_games,
                "goalie_stats_gamesStarted" : goalie_gamesStarted,
                "goalie_stats_shotsAgainst" : goalie_shotsAgainst,
                "goalie_stats_goalie_goalsAgainst" : goalie_goalsAgainst,
                "goalie_stats_timeOnIcePerGame" : goalie_timeOnIcePerGame,
                "goalie_stats_powerPlaySavePercentage" : goalie_powerPlaySavePercentage,
                "goalie_stats_shortHandedSavePercentage" : goalie_shortHandedSavePercentage,
                "goalie_stats_evenStrengthSavePercentage" : goalie_evenStrengthSavePercentage,

           })
           
goalie_statistics = pd.DataFrame(goalie_stats)
goalie_statistics.to_csv("./schema/goalie_statistics_utf-8.csv", index=False, encoding = 'utf-8-sig')
goalie_statistics.to_csv("./schema/goalie_statistics.csv", index=False)
goalie_statistics.head()
      

PermissionError: [Errno 13] Permission denied: './schema/goalie_statistics_utf-8.csv'

Final Analysis/Conclusion:

- Based on our overall analysis and supporting data, there is no material difference regarding age groups and winning.  Years of experience requires additional analysis.
- There should be a heightened focus on recruiting European players.  Their hockey programs have significantly evolved, and the number of European players playing in the NHL is showing a steady increase, while the Canadian player representation is reducing.
- More PPG equates to a higher overall point value and not a substitute for regular goals.
- It is favorable and improves the likelihood of winning to have more consistency amongst contributing players vs having one or two superstars on the team.  The box plot comparing the +/- statistic supports this theor as the winning team has less outliers and more consistency when comparing the winning and losing team.  The +/- statistic is also the only key statistic analyzed that has a material difference(100% increase) in results when comparing the winning and losing team.
- Future analysis reviewing Stanley Cup Finals' data only, can provide greater variability in data points and provide additional trending, influencing additional recommendations.